In [12]:
from pathlib import Path
from scipy.io import wavfile
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from math import floor, ceil
from itertools import islice
# from commpy.modulation import QAMModem
# from commpy.filters import rrcosfilter
import sys
from functools import reduce
np.set_printoptions(threshold=sys.maxsize)

## Resources

ETSI standard (*ETSI TS 102 361*):

1. [Air interface protocol](https://www.dmrassociation.org/downloads/standards/ts_10236101v020501p.pdf)
2. [DMR voice and generic services](https://www.dmrassociation.org/downloads/standards/ts_10236102v020401p.pdf)
3. [Data protocol](https://www.dmrassociation.org/downloads/standards/ts_10236103v010301p.pdf)
4. [Trunking protocol](https://www.dmrassociation.org/downloads/standards/ts_10236104v011001p.pdf)

[DMRDecode](https://github.com/IanWraith/DMRDecode) (Java)

[mbelib](https://github.com/szechyjs/mbelib) (C++)

## Asset / Example IQ File

[SDRSharp_20160101_231914Z_12kHz_IQ.wav](https://www.sigidwiki.com/wiki/File:DMR.zip)

Source: https://www.sigidwiki.com/wiki/Digital_Mobile_Radio_(DMR)



In [2]:
SYMBOL_RATE = 4800
SYMBOL_TIME = 1 / SYMBOL_RATE
PACKET_LENGTH_MS = 0.03
PACKET_LENGTH_SYMBOLS = 132
CACH_BURST_LENGTH_SYMBOLS = 12
PACKET_LENGTH_BITS = PACKET_LENGTH_SYMBOLS * 2

In [3]:
# ETSI TS 102 361-1 Section 9.1.1
SYNC_PATTERN = {
    "BS_VOICE":                 "011101010101111111010111110111110111010111110111",
    "BS_DATA":                  "110111111111010101111101011101011101111101011101",
    "MS_VOICE":                 "011111110111110101011101110101010111110111111101",
    "MS_DATA":                  "110101011101011111110111011111111101011101010111",
    "MS_RC_SYNC":               "011101111101010101011111011111011111110101110111",
    "TDMA_DIRECT_SLOT_1_VOICE": "010111010101011101111111011101110101011111111111",
    "TDMA_DIRECT_SLOT_1_DATA":  "111101111111110111010101110111011111110101010101",
    "TDMA_DIRECT_SLOT_2_VOICE": "011111011111111111010101111101010101110101011111",
    "TDMA_DIRECT_SLOT_2_DATA":  "110101110101010101111111010111111111011111110101",
    "RESERVED_SYNC_PATTERN":    "110111010111111111110101110101110101011111011101"
}

SYNC_PATTERN_DECIMAL = {
     key: int(value, 2) for (key, value) in SYNC_PATTERN.items()
}


# On error-correcting codes: http://www.math.toronto.edu/afenyes/writing/error-correction%20(october%202015).pdf

# ETSI TS 102 361-1 Section B.3.5
HAMMING_7_4_3_GENERATOR = np.array([
    [1, 0, 0, 0, 1, 0, 1],
    [0, 1, 0, 0, 1, 1, 1],
    [0, 0, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 1]
])
# https://www.slideserve.com/elijah/eee436 Slide 5
HAMMING_7_4_3_PARITY_CHECK = np.array([
    [1, 0, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1]
]).transpose()

In [4]:
def load_asset(name):
    base_path = Path(globals()['_dh'][0]).parent
    file_path = (base_path / "assets" / name).resolve()
    sample_rate, file = wavfile.read(file_path)
    print(f'Loaded {file_path}')
    num_channels = len(file.shape)
    length_s = round(file.shape[0]/sample_rate, 2)
    print(f'Sample rate: {sample_rate}Hz, channels: {num_channels}, samples: {file.shape[0]}, length: {length_s}s')
    
    return (sample_rate, file)


def save_wav(file, fs, file_path='/tmp/test.wav'):
    # normalize volume
    frames = (file * (20000 / np.max(file))).astype('int16')

    w = wave.open(file_path, mode='wb')
    w.setnchannels(1)
    w.setframerate(fs)
    w.setsampwidth(2)
    w.writeframes(frames)
    w.close()
    
    print(f'Saved file to {file_path} - {round(len(file)/fs, 2)}s @ {fs}Hz')

def show_waveform(file, fs, comment=''):
    fig, ax = plt.subplots()
    ax.plot([ (n*1000)/fs for n in range(0, len(file)) ], file)
    ax.set_title(f'Waveform - {comment}')
    
    for offset in range(0, floor((len(file)/fs)/SYMBOL_TIME)+1, 2):
        ax.axvspan(offset*SYMBOL_TIME * 1000, (offset + 1) * SYMBOL_TIME * 1000, alpha=0.2, color='pink')

    fig.set_figwidth(20)
    ax.set_xlabel('Time [ms]')
    ax.set_ylabel('Amplitude')
    

def show_spectrogram(x, fs, comment=''):
    f, t, Sxx = signal.spectrogram(x, fs)

    fig, ax = plt.subplots()
    ax.pcolormesh(t, f, Sxx, shading='gouraud')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
#     ax.axhspan(1200-10, 1200+10, alpha=0.2, color='pink')
#     ax.axhspan(2200-10, 2200+10, alpha=0.2, color='pink')
    fig.set_figwidth(20)
    ax.set_ylim([0,8000])
    ax.set_title(f'Spectrogram - {comment}')

In [5]:
class RootRaisedCosineFilter:
    # Translated from the DMRDecode project https://github.com/IanWraith/DMRDecode

    def __init__(self):
        self.N_ZEROS = 80
        self.xv = [ 0.0 for i in range(0, self.N_ZEROS+2) ]
        self.xv_counter = 0

        self.GAIN = 9.868410946e+00
        self.X_COEFFS = [
            +0.0273676736, +0.0190682959, +0.0070661879, -0.0075385898,
            -0.0231737159, -0.0379433607, -0.0498333862, -0.0569528373,
            -0.0577853377, -0.0514204905, -0.0377352004, -0.0174982391,
            +0.0076217868, +0.0351552125, +0.0620353691, +0.0848941519,
            +0.1004237235, +0.1057694293, +0.0989127431, +0.0790009892,
            +0.0465831968, +0.0037187043, -0.0460635022, -0.0979622825,
            -0.1462501260, -0.1847425896, -0.2073523972, -0.2086782295,
            -0.1845719273, -0.1326270847, -0.0525370892, +0.0537187153,
            +0.1818868577, +0.3256572849, +0.4770745929, +0.6271117870,
            +0.7663588857, +0.8857664963, +0.9773779594, +1.0349835419,
            +1.0546365475, +1.0349835419, +0.9773779594, +0.8857664963,
            +0.7663588857, +0.6271117870, +0.4770745929, +0.3256572849,
            +0.1818868577, +0.0537187153, -0.0525370892, -0.1326270847,
            -0.1845719273, -0.2086782295, -0.2073523972, -0.1847425896,
            -0.1462501260, -0.0979622825, -0.0460635022, +0.0037187043,
            +0.0465831968, +0.0790009892, +0.0989127431, +0.1057694293,
            +0.1004237235, +0.0848941519, +0.0620353691, +0.0351552125,
            +0.0076217868, -0.0174982391, -0.0377352004, -0.0514204905,
            -0.0577853377, -0.0569528373, -0.0498333862, -0.0379433607,
            -0.0231737159, -0.0075385898, +0.0070661879, +0.0190682959,
            +0.0273676736
        ]
    
    def filter_sample(self, sample):
        sum = 0.0

        # Add the latest sample to the xv circular buffer
        self.xv[self.xv_counter] = (float(sample) / self.GAIN)

        # Increment the circular buffer counter and zero it if needed
        self.xv_counter = self.xv_counter + 1
        if self.xv_counter == (self.N_ZEROS+1):
            self.xv_counter = 0

        # Do the RRC maths taking account of the fact that XV is a circular buffer
        xvShadow = self.xv_counter

        for i in range(0, self.N_ZEROS + 1):
            sum = sum + (self.X_COEFFS[i] * self.xv[xvShadow])
            xvShadow = xvShadow + 1
            if xvShadow == (self.N_ZEROS + 1):
                xvShadow=0

        return int(sum)

In [6]:
fs_real, rec_real = load_asset('SDRSharp_20160101_231914Z_12kHz_real.wav')
rrcos = RootRaisedCosineFilter()

fs = 48000
SAMPLES_PER_SYMBOL = int(fs / SYMBOL_RATE)

rec_real_resampled = signal.resample(rec_real, int((fs/fs_real) * len(rec_real)))

rec_real_resampled_filtered = np.array([ rrcos.filter_sample(b) for b in rec_real_resampled ])

#Audio(rec_real_resampled, autoplay=False, rate=fs)
#Audio(rec_real_resampled_filtered, autoplay=False, rate=fs)

Loaded /home/thomas/code/dmr-from-scratch/assets/SDRSharp_20160101_231914Z_12kHz_real.wav
Sample rate: 44100Hz, channels: 1, samples: 1823133, length: 41.34s


In [7]:
# First order of business: Syncing phase
# We want to do this repeatedly to handle phase wander

def find_best_phase_offset(wave, samples_per_symbol):
    # We use the standard deviation as a heuristic for the "how nicely are they split into bins?"
    # Seems to work well enough

    best_offset = np.argmax([
        np.std(wave[offset::samples_per_symbol])
        for offset in range(0, samples_per_symbol)
    ])
    
    return best_offset

# Allows to visualize different phase offsets
# We want the one where samples are divided neatly in 4 bins
def visualize_phase_offsets(wave, samples_per_symbol):
    for offset in range(0, samples_per_symbol):
        fig, ax = plt.subplots()
        ax.hist(wave[offset::samples_per_symbol], bins=100)
        ax.set_title(f'Phase offset {offset}')


def bit_string_to_symbols(bitstring):
    return np.array([
        { '00': +1, '01': +3, '10': -1, '11': -3 }[b1 + b2]
        for b1, b2 in zip(bitstring[::2], bitstring[1::2])
    ])

def bit_array_to_decimal(bit_array):
    return reduce(lambda acc, curr: (acc << 1) + curr, bit_array, 0)

def symbol_to_dibit(input):
    return {
        3: "01", # 1, # 01
        1: "00", # 0, # 00
        -1: "10", # 2, # 10
        -3: "11", # 3  # 11
    }[input]

def symbol_to_decimal(symbol):
    return {
        3: 1,
        1: 0,
        -1: 2,
        -3: 3,
    }[symbol]

def four_symbols_to_byte(symbols):
    return ((symbol_to_decimal(symbols[0]) << 6) +
        (symbol_to_decimal(symbols[1]) << 4) +
        (symbol_to_decimal(symbols[2]) << 2) +
        symbol_to_decimal(symbols[3]))

def symbols_to_int(symbols):
    return int.from_bytes(bytes([
        four_symbols_to_byte(four_dibits) for four_dibits in np.split(symbols, len(symbols)/4)
    ]), byteorder='big')

# np.digitize will assign the number of the bin
# This map maps bin numbers to dibits
# See ETSI TS 102 361-1 Table 10.3
DIGITIZED_TO_SYMBOL = {
    1: +3, # 01
    2: +1, # 00
    3: -1, # 10
    4: -3  # 11
}

# SYNC_PATTERN_SYMBOLS = {
#     key: bit_string_to_symbols(value) for (key, value) in SYNC_PATTERN.items()
# }

# Split the input in parts of ~1s each, find the best phase offset for each part, then sample
split_per_second = np.array_split(
    rec_real_resampled_filtered,
    1 # TODO ceil(len(rec_real_resampled_filtered)/fs)
)

sampled = np.array([
    list(map(lambda sec: sec[find_best_phase_offset(sec, SAMPLES_PER_SYMBOL)::SAMPLES_PER_SYMBOL], split_per_second))
]).flatten()

# sampled = np.array([
#     (point for point in sec[find_best_phase_offset(sec, SAMPLES_PER_SYMBOL)::SAMPLES_PER_SYMBOL])
#     for sec in split_per_second
# ])

# Sample the waveform, using the found optimal phase offset
# sampled = rec_real_resampled_filtered[phase_offset::SAMPLES_PER_SYMBOL]

# Generate bins to sort samples
middle = (np.quantile(sampled,0.05) + np.quantile(sampled,0.95)) / 2
bins = [np.min(sampled)-1, (np.min(sampled)+middle)/2, middle, (np.max(sampled)+middle)/2, np.max(sampled)+1]

# Bin the samples and convert them to a bitstream
symbol_stream = np.array([ DIGITIZED_TO_SYMBOL[s] for s in np.digitize(sampled, bins, right=True) ])

def is_burst_with_sync_pattern(symbols):
    packet_sync_region = symbols[54:54+24]
    
    if len(packet_sync_region) != 24:
        return False

    return symbols_to_int(packet_sync_region) in SYNC_PATTERN_DECIMAL.values()

def rolling_window(a, window):
    # TODO just use np.lib.stride_tricks.sliding_window_view(a, window) instead of this function?
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

burst_starts = np.where([ is_burst_with_sync_pattern(sub) for sub in rolling_window(symbol_stream, PACKET_LENGTH_SYMBOLS) ])[0]

# bursts = [ symbol_stream[burst_start:burst_start+PACKET_LENGTH_SYMBOLS] for burst_start in burst_starts ]

In [13]:
symbol_stream

array([-1, -1, -1, -1,  1,  1,  3, -1,  1,  3, -1, -3,  1, -1, -3, -3, -1,
        3,  1,  3, -1, -1,  1,  1, -1,  3, -3,  1, -1, -3,  1, -3, -3, -3,
       -3, -1,  3, -1,  1, -1, -3,  1, -3, -1,  1, -3, -1, -1,  1, -3, -3,
       -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,  3, -1,  1,  3, -1, -1,  1,
       -3, -3, -1, -1, -3, -3,  1,  1, -1,  1,  1,  3, -1,  3, -1,  3, -1,
       -1, -3, -3, -1,  1,  1, -3, -1, -3, -3, -1,  3, -1,  3, -1, -1, -1,
       -1,  3,  1, -3, -1, -1,  3, -3, -1,  1, -1, -1,  3, -3,  1,  1,  1,
        3,  1,  3, -1,  1,  1,  1, -1,  3,  3, -1, -3, -1,  1,  3, -1,  3,
       -3,  1,  1, -1,  3,  1,  3,  1,  1, -3, -3,  1,  1, -3, -3, -1,  3,
        1,  1,  1, -3, -3, -1,  3,  3, -1, -1,  1, -3, -1,  3,  1,  1,  1,
        1,  3,  1,  1,  1, -1, -1,  1, -3,  3, -3, -1,  1, -3,  3, -3, -1,
        3, -1,  3,  1, -1, -1,  3,  1,  3,  1, -3,  1, -3,  1,  1, -3,  1,
       -1, -1,  1,  1, -3,  1,  1, -3,  3,  3,  3,  1, -3, -3, -3,  3,  1,
       -3, -1,  3, -3, -1

In [8]:
BS_DATA_TYPE = {
    0: 'PI Header',
    1: 'Voice LC Header',
    2: 'Terminator with LC',
    3: 'CSBK',
    4: 'MBC Header',
    5: 'MBC Continuation',
    6: 'Data Header',
    7: 'Rate ½ Data Continuation',
    8: 'Rate ¾ Data Continuation',
    9: 'Idle',
    10: 'Unified Single Block Data',
    11: 'Data Type 11 - UNDEFINED',
    12: 'Data Type 12 - UNDEFINED',
    13: 'Data Type 13 - UNDEFINED',
    14: 'Data Type 14 - UNDEFINED',
    15: 'Data Type 15 - UNDEFINED'
}

class BSDataBurst:
    def __init__(self, raw, info, colour_code, data_type, fec_parity, sync, slot):
        self.raw = raw
        self.info = info
        self.colour_code = colour_code
        self.data_type = BS_DATA_TYPE[data_type]
        self.fec_parity = fec_parity
        self.sync = sync
        self.slot = slot
        
    def __repr__(self):
        return f'BS data burst - slot {hex(self.slot)}, info {hex(self.info)},  colour code {self.colour_code}, data type {self.data_type}, fec {hex(self.fec_parity)}, sync {hex(self.sync)} [raw {hex(self.raw)}]'
    
    def create_from_burst_binary(data):
        MASK_DATA_1 =    0xffffffffffffffffffffffffc00000000000000000000000000000000000000000
        MASK_CC =        0x3c0000000000000000000000000000000000000000
        MASK_DATA_TYPE = 0x3c000000000000000000000000000000000000000
        MASK_FEC_1 =     0x3000000000000000000000000000000000000000
        MASK_SYNC =      0xffffffffffff000000000000000000000000000
        MASK_FEC_2 =     0xffc000000000000000000000000
        MASK_DATA_2 =    0x3ffffffffffffffffffffffff
        
        # TODO get cc, data type, and fec from slot 
        MASK_SLOT_1 = 0x3ff000000000000000000000000000000000000000
        MASK_SLOT_2 = 0xffc000000000000000000000000
        slot = ((data & MASK_SLOT_1) >> 146) + ((data & MASK_SLOT_2) >> 98)
        
        sync = (data & MASK_SYNC) >> 108
        
        if sync != SYNC_PATTERN_DECIMAL['BS_DATA']:
            raise ValueError('BSDataBurst.create_from_burst_binary: Sync is not applicable for BS data burst')
        
        return BSDataBurst(
            data,
            ((data & MASK_DATA_1) >> 68) + (data & MASK_DATA_2),
            (data & MASK_CC) >> 162,
            (data & MASK_DATA_TYPE) >> 158,
            ((data & MASK_FEC_1) >> 146) + ((data & MASK_FEC_2) >> 98),
            sync,
            slot
        )
        

class BSVoiceBurst:
    def __init__(self, raw, voice, sync):
        self.raw = raw
        self.voice = voice
        self.sync = sync
        
    def __repr__(self):
        return f'BS voice burst - voice data {hex(self.voice)}, sync {hex(self.sync)} [raw {hex(self.raw)}]'
    
    def create_from_burst_binary(data):
        MASK_VOICE_1 = 0xfffffffffffffffffffffffffff000000000000000000000000000000000000000
        MASK_VOICE_2 = 0xfffffffffffffffffffffffffff
        MASK_SYNC    = 0xffffffffffff000000000000000000000000000
        
        sync = (data & MASK_SYNC) >> 108
        
        if sync != SYNC_PATTERN_DECIMAL['BS_VOICE']:
            raise ValueError('BSVoiceBurst.create_from_burst_binary: Sync is not applicable for BS voice burst')
        
        return BSVoiceBurst(
            data,
            ((data & MASK_VOICE_1) >> 48) + (data & MASK_VOICE_2),
            sync
        )

tmp_good_emb = [
    0,627,1253,1686,2505,3002,3372,3935,4578,5009,5383,6004,6187,
    6744,7374,7869,8631,9156,9554,10017,10366,10765,11419,12008,12373,12838,13488, 
    14019,14748,15343,15737,16138,16670,17261,17915,18312,18647,19108,19506,20033, 
    20732,21135,21529,22122,22837,23366,24016,24483,24745,25306,25676,26175,26976, 
    27411,28037,28662,29003,29496,30126,30685,30850,31473,31847,32276,32847,33340, 
    33962,34521,35206,35829,36195,36624,37293,37854,38216,38715,39012,39447,40065, 
    40690,41464,41867,42269,42862,43057,43586,44244,44711,45082,45673,46335,46732, 
    47571,48032,48438,48965,49489,49954,50612,51143,51352,51947,52349,52750,53427, 
    53952,54358,54821,55674,56073,56735,57324,57574,58005,58371,58992,59695,60252, 
    60874,61369,61700,62327,62945,63378,63693,64190,64552,65115
]
    
class BSVoiceBurstWithEmbeddedSignaling:
    def __init__(self, raw, voice, embedded_signaling, emb_parity, lcss, pi, cc, emb_raw):
        self.raw = raw
        self.voice = voice
        self.embedded_signaling = embedded_signaling
        self.emb_parity = emb_parity
        self.lcss = lcss
        self.pi = pi
        self.cc = cc
        self.emb_raw = emb_raw # TODO tmp
        
    def __repr__(self):
        return f'Voice burst with embedded signaling - {self.emb_raw in tmp_good_emb}, embedded signaling {self.embedded_signaling}, emb_raw {self.emb_raw}, emb parity {self.emb_parity} [ raw {hex(self.raw)} ]'
    
    def create_from_burst_binary(data):
        MASK_VOICE_1 = 0xfffffffffffffffffffffffffff000000000000000000000000000000000000000
        MASK_VOICE_2 = 0xfffffffffffffffffffffffffff
        MASK_EMB_1 = 0xff0000000000000000000000000000000000000
        MASK_EMB_2 = 0xff000000000000000000000000000
        MASK_EMBEDDED_SIGNALING = 0xffffffff00000000000000000000000000000
        
        emb = ((data & MASK_EMB_1) >> 140) + ((data & MASK_EMB_2) >> 108)
        
        emb_parity = emb & 0x1ff
        lcss = (emb >> 9) & 0x3 
        pi = emb >> 11
        cc = (emb >> 12) & 0xf
        
        
        return BSVoiceBurstWithEmbeddedSignaling(
            data,
            ((data & MASK_VOICE_1) >> 48) + (data & MASK_VOICE_2),
            ((data & MASK_EMBEDDED_SIGNALING) >> 116),
            emb_parity,
            lcss,
            pi,
            cc,
            emb
        )
    

def sync_pattern_to_name(sync_pattern):
    for (key, value) in SYNC_PATTERN_DECIMAL.items():
        if np.array_equal(sync_pattern, value):
            return key

def create_burst_from_symbols(symbols):
    packet_sync_region = symbols[54:54+24]
    
    name = sync_pattern_to_name(symbols_to_int(packet_sync_region))
    bits = symbols_to_int(symbols)
    
    if name == 'BS_DATA':
        return BSDataBurst.create_from_burst_binary(bits)
    elif name == 'BS_VOICE':
        return BSVoiceBurst.create_from_burst_binary(bits)
    else:
        return None
        return BSVoiceBurstWithEmbeddedSignaling.create_from_burst_binary(bits)
    
# [ create_burst_from_symbols(burst) for burst in bursts ]

In [25]:
symbol_stream[burst_starts[0]-1000:burst_starts[1] - 1000]

array([ 1, -1, -1, -1, -3, -1, -3,  3, -1,  1, -1,  1, -3, -3, -3, -3,  3,
       -1, -3, -1, -1, -1,  3,  1,  3,  1, -3,  1,  1, -1,  1, -3,  3,  1,
       -1, -1,  1,  3, -3, -1,  3,  1, -3, -3,  1, -3, -1,  1, -3, -3, -1,
       -1, -3, -3, -1, -3,  3,  1, -1, -3, -1, -3, -1, -3, -1, -3, -3, -3,
       -1,  1, -3,  1,  1,  3, -1, -3, -3,  3,  1, -3, -1, -1, -3, -1,  3,
        1,  1, -1,  3, -1, -1,  1,  1, -3,  3, -1,  1,  1, -1,  1, -3, -1,
       -1,  3, -1, -3,  1, -1, -3,  1, -1,  3,  3, -3,  3,  3, -1,  1,  3,
        1,  3, -3,  3, -1,  1,  1, -1, -1,  1, -1,  3, -1, -1, -1, -1, -1,
       -3, -3, -1,  3,  3, -1, -1,  1, -3, -3,  3,  1,  1,  1,  1,  3,  1,
        1,  1, -1, -3,  1, -3,  3, -1, -3,  3, -3,  3, -1, -3,  3, -1,  3,
        1, -1, -1,  3,  1,  3,  3, -3,  1, -3, -1,  3, -3,  1, -1, -1,  1,
        1, -3, -1,  3, -3,  3,  3,  3,  3, -3, -3, -3,  3,  3, -3, -3,  3,
       -1, -3,  3, -3,  3,  3, -3, -3,  3, -3,  3,  3,  1, -1, -1,  3,  3,
        3, -3,  1,  3,  1

In [9]:
# Example burst 24722691057115041339106602335442372438215333147166828539499156192919342482718855
# EMB (emb_raw) 0b0010001111000100 (9156) => passing parity


a = BSVoiceBurstWithEmbeddedSignaling.create_from_burst_binary(24722691057115041339106602335442372438215333147166828539499156192919342482718855)
a

Voice burst with embedded signaling - True, embedded signaling 100994054, emb_raw 9156, emb parity 452 [ raw 0xd5826342d9f5cf1a2fc6b24264e2306050c06c4bf49a591bc18544608d7ccc0087 ]

In [22]:

def create_bursts(data): # TODO better name
    # Given a range starting with packet with sync, create (both data/voice and CACH) bursts
    
    # The range has [ channel burst, cach, channel burst, cach, channel burst, ... ]
    # We'll call [ channel burst, cach ] a "pair" here

    # TODO it's better to do this in revers ([cach, burst]) as cach contains information about coming frame
    
    num_pairs = floor(len(data) / (PACKET_LENGTH_SYMBOLS + CACH_BURST_LENGTH_SYMBOLS))
        
    data = data[0:num_pairs*(PACKET_LENGTH_SYMBOLS + CACH_BURST_LENGTH_SYMBOLS)]
    data = np.split(data, num_pairs)
        
    return [
        burst for pair in data for burst in [
            create_burst_from_symbols(pair[:PACKET_LENGTH_SYMBOLS]),
            ReceivedCachBurst.create_from_burst_binary(symbols_to_int(pair[PACKET_LENGTH_SYMBOLS:]))
        ]
    ]


[
    item
    for (burst_start, next_burst_start) in list(zip(burst_starts, burst_starts[1:]))
    for item in create_bursts(symbol_stream[burst_start:next_burst_start])
]

[BS voice burst - voice data 0xf968f40102cdb76d9291739ca0e5295c4ecfbdb1e260c7e56cba1e, sync 0x755fd7df75f7 [raw 0xf968f40102cdb76d9291739ca0e755fd7df75f75295c4ecfbdb1e260c7e56cba1e],
 CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 None,
 CACH burst (FEC valid): at 0, tc 1, ls 0x0, payload 0x12e4 [raw (deinterleaved) 0x4e12e4],
 None,
 CACH burst (FEC valid): at 1, tc 0, ls 0x0, payload 0x9089 [raw (deinterleaved) 0x8a9089],
 None,
 CACH burst (FEC valid): at 0, tc 1, ls 0x1, payload 0xf7d1 [raw (deinterleaved) 0x58f7d1],
 None,
 CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 None,
 CACH burst (FEC valid): at 0, tc 1, ls 0x0, payload 0x12a4 [raw (deinterleaved) 0x4e12a4],
 None,
 CACH burst (FEC valid): at 1, tc 0, ls 0x0, payload 0x9089 [raw (deinterleaved) 0x8a9089],
 None,
 CACH burst (FEC valid): at 0, tc 1, ls 0x1, payload 0xf7d1 [raw (deinterleaved) 0x58f7d1],
 None,
 CACH burst (FEC valid): 

In [16]:
# ETSI TS 102 361-1 B.4.1
def deinterleave_cach_burst(cach_burst):
    CACH_BURST_DEINTERLEAVE_TABLE = [0, 4, 8, 12, 14, 18, 22, 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 15, 16, 17, 19, 20, 21, 23]

    return [
        cach_burst[i]
        for i in CACH_BURST_DEINTERLEAVE_TABLE
    ]

# ETSI TS 102 361-1 B.4.1
def interleave_cach_burst(cach_burst):
    CACH_BURST_INTERLEAVE_TABLE = [0, 7, 8, 9, 1, 10, 11, 12, 2, 13, 14, 15, 3, 16, 4, 17, 18, 19, 5, 20, 21, 22, 6, 23]

    return [
        cach_burst[i]
        for i in CACH_BURST_INTERLEAVE_TABLE
    ]


# deinterleaved_cach_burst_example = [ 'AT', 'P16', 'P15', 'P14', 'TC', 'P13', 'P12', 'P11', 'LS1', 'P10', 'P9', 'P8', 'LS0', 'P7', 'H2', 'P6', 'P5', 'P4', 'H1', 'P3', 'P2', 'P1', 'H0', 'P0' ]
# interleave_cach_burst(deinterleave_cach_burst(deinterleaved_cach_burst_example))

# Purpose of CACH (ETSI TS 102 361-1 Section 4.5):
#  1. Indicate the usage of the inbound time slot
#  2. Indicate the channel number of the inbound and outbound time slots
#  3. Carry additional low speed signalling as described in clause 7.1.4

class ReceivedCachBurst:
    # at      : access (1 bit)               \
    # tc      : numbering (1 bit)            |
    # ls/lcss : framing (2 bits)             |     Known together as "TDMA Access Channel Type" (TACT) bits
    # h       : hamming parity bits (3 bits) /
    # p       : payload (17 bits)            --> Payload NOT protected by Hamming FEC
    def __init__(self, raw, access, numbering, framing, hamming, payload):
        self.raw = raw

        # Interpretation of at and tc (ETSI TS 102 361-1 Section 6.3):
        #  Where DMR activity is present on the outbound channel,
        #  then the AT bit in each CACH shall indicate to MSs whether
        #  the next slot on the inbound channel whose TDMA channel number
        #  is indicated by the TC bit is "Idle" or "Busy"
        # Typically a BS shall set the AT to "Busy" while DMR activity is present on the inbound channel
        self.access = access
        self.numbering = numbering
        
        # LCSS indicates that this burst contains the beginning, end, or continuation of an LC or CSBK signalling
        # Due to the small number of bits available, there is no single fragment LC signalling defined
        self.framing = framing
        
        self.hamming = hamming
        
        self.payload = payload
        self.tact = np.array([
            access,
            numbering,
            (framing & 0x2) >> 1,
            (framing & 0x1),
            (hamming & 0x4) >> 2,
            (hamming & 0x2) >> 1,
            (hamming & 0x1),
        ])

    def __repr__(self):
        return f'CACH burst (FEC { "" if self.has_valid_fec() else "in" }valid): at {self.access}, tc {self.numbering}, ls {hex(self.framing)}, payload {hex(self.payload)} [raw (deinterleaved) {hex(self.raw)}]'

    # Only TACT is protected by FEC
    def has_valid_fec(self):
        return np.array_equal((self.tact @ HAMMING_7_4_3_PARITY_CHECK) % 2, np.array([0, 0, 0]))
    
    def create_from_burst_binary(data):
        deinterleaved = deinterleave_cach_burst([ int(i) for i in bin(data)[2:].zfill(24) ])
        
        return ReceivedCachBurst(
            bit_array_to_decimal(deinterleaved),
            deinterleaved[0],
            deinterleaved[1],
            (deinterleaved[2] << 1) + deinterleaved[3],
            (deinterleaved[4] << 2) + (deinterleaved[5] << 1) + deinterleaved[6],
            bit_array_to_decimal(deinterleaved[7:])
        )
    

    
a = ReceivedCachBurst.create_from_burst_binary(11977059)
# a.has_valid_fec()
a


[
    ReceivedCachBurst.create_from_burst_binary(
        symbols_to_int(symbol_stream[i+PACKET_LENGTH_SYMBOLS:i+PACKET_LENGTH_SYMBOLS+12])
    ) for i in burst_starts[:-1]
]

[CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 CACH burst (FEC valid): at 1, tc 0, ls 0x0, payload 0x9089 [raw (deinterleaved) 0x8a9089],
 CACH burst (FEC valid): at 1, tc 0, ls 0x2, payload 0xf451 [raw (deinterleaved) 0xa6f451],
 CACH burst (FEC valid): at 0, tc 1, ls 0x1, payload 0xf7d1 [raw (deinterleaved) 0x58f7d1],
 CACH burst (FEC valid): at 0, tc 1, ls 0x1, payload 0xf7d1 [raw (deinterleaved) 0x58f7d1],
 CACH burst (FEC valid): at 0, tc 1, ls 0x0, payload 0x12e4 [raw (deinterleaved) 0x4e12e4],
 CACH burst (FEC valid): at 0, tc 1, ls 0x1, payload 0xf7d1 [raw (deinterleaved) 0x58f7d1],
 CACH burst (FEC valid): at 0, tc 1, ls 0x0, payload 0x12e4 [raw (deinterleaved) 0x4e12e4],
 CACH burst (FEC valid): at 1, tc 0, ls 0x0, payload 0x9089 [raw (deinterleaved)

In [32]:
deinterleave_cach_burst([1,0,1,0,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,1,0,0,0,1])

[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1]

In [17]:
Example CACH bursts with deinterleaved data and TACT

Raw CACH                 Deinterleaved CACH
------------------------ ------------------------
000010100010011110101010 010011100001001011100100
101000100000011000011011 100010101001000010001101
001111100111111101000001 010110001111011111010001
101101101100000101100011 101001101111010001010001
000010100010011110101010 010011100001001011100100
101000100000011000010011 100010101001000010001001
001111100111111101000001 010110001111011111010001
101101101100000101100011 101001101111010001010001
000010100010011110101010 010011100001001011100100
101000100000011000010011 100010101001000010001001
001111100111111101000001 010110001111011111010001
101101101100000101100011 101001101111010001010001
000010100010011110101010 010011100001001011100100
101000100000011000010011 100010101001000010001001
001111100111111101000001 010110001111011111010001
101101101100000101100011 101001101111010001010001
                         ^^^^^^^
                          TACT
        
        
tact = deinterleave_cach_burst([ int(b) for b in '000111000110101111001001' ])[0:7]

tact @ HAMMING_7_4_3_PARITY_CHECK % 2

[0,0,0] => no errors

SyntaxError: invalid syntax (<ipython-input-17-eaee924e77e7>, line 1)

In [28]:
ReceivedCachBurst.create_from_burst_binary(665514)

CACH burst (FEC valid): at 0, tc 1, ls 0x0, payload 0x12e4 [raw (deinterleaved) 0x4e12e4]